Week1: Generic time series function creation and keras metric

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import sklearn
from matplotlib import style
style.use('seaborn')
tf.enable_eager_execution()

def plot_series(time, series, format = '-', start = 0 , end = None):
    plt.plot(time[start:end],series[start:end], format)
    plt.xlabel('time')
    plt.ylabel('values')
    plt.grid(True)

def trend(time, slope = 0):
    return slope*time

def seasonal_pattern(season_time):
    return np.where(season_time<0.4,
                    np.cos(season_time)*5,
                    np.exp(season_time))

def seasonality(time, period, phase = 0, amplitude = 1):
    season_time = ((time + phase) % period) / period
    return  seasonal_pattern(season_time) * amplitude

def noise(time, noise_level = 1, seed = None):
    rnd = np.random.RandomState(seed)
    return rnd.rand(len(time)) * noise_level

time = np.arange(4*365+1, dtype = 'float')
baseline = 10
amplitude = 40
slope = 0.2
noise_level = 5
series = baseline + trend(time,slope) + seasonality(time = time,period = 365, phase = 0,amplitude = amplitude) + noise(time, noise_level = noise_level, seed = 1)


plt.figure(figsize = (10,6))
plot_series(time,series)

from sklearn.model_selection import train_test_split
time_train, time_test, series_train, series_test = train_test_split(time, series, test_size=0.33, shuffle = False)

naive_forecast = series[len(time_train)-1:-1]
plt.figure('test', figsize = (10,6))
plot_series(time_test,series_test,end = 150)
plot_series(time_test,naive_forecast, end = 150)

print('mae_naive' , keras.metrics.mean_absolute_error(series_test,naive_forecast).numpy())
print('mse_naive' , keras.metrics.mean_squared_error(series_test,naive_forecast).numpy())

def moving_average_forecast(series, window_size = 1):
    forecast = []
    for i in range(len(series) - window_size):
        forecast.append(np.mean(series[i:i+window_size]))
    return np.array(forecast)

moving_average = moving_average_forecast(series, 30)[len(time_train)-30:]
plt.figure('moving_average', figsize = (10,6))
plot_series(time_test, series_test)
plot_series(time_test,moving_average)

print('mae_moving',keras.metrics.mean_absolute_error(series_test, moving_average).numpy())
print('mse_moving',keras.metrics.mean_squared_error(series_test, moving_average).numpy())

Week2

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import sklearn
from matplotlib import style
style.use('seaborn')
tf.enable_eager_execution()

dataset = tf.data.Dataset.range(10)

for tensors in dataset:
    print(tensors)

dataset = dataset.window(5, shift = 1,stride = 1, drop_remainder = True)
for window_dataset in dataset:
    for val in window_dataset:
        print(val.numpy(), end = '-')
    print()

dataset = dataset.flat_map(lambda window: window.batch(5))
for window in dataset:
    print(window.numpy())

dataset = dataset.map(lambda window: (window[:-1],window[-1:]))
for x,y in dataset:
    print(x.numpy(),y.numpy())

dataset = dataset.shuffle(buffer_size = 10)
for x,y in dataset:
    print(x.numpy(),y.numpy())

dataset = dataset.batch(2).prefetch(1)
for x,y in dataset:
    print(x.numpy(),y.numpy())

def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    dataset = tf.data.Dataset.from_tensor_slices(series)
    dataset = dataset.window(window_size + 1, shift = 1,drop_remainder = True)
    dataset = dataset.flat_map(lambda window: window.batch(window_size+1))
    dataset = dataset.shuffle(shuffle_buffer)
    dataset = dataset.map(lambda window: (window[:-1],window[-1]))
    dataset = dataset.batch(batch_size).prefetch(1).repeat()
    return dataset

def plot_series(time, series, format = '-', start = 0 , end = None):
    plt.plot(time[start:end],series[start:end], format)
    plt.xlabel('time')
    plt.ylabel('values')
    plt.grid(True)

def trend(time, slope = 0):
    return slope*time

def seasonal_pattern(season_time):
    return np.where(season_time<0.4,
                    np.cos(season_time)*5,
                    np.exp(season_time))

def seasonality(time, period, phase = 0, amplitude = 1):
    season_time = ((time + phase) % period) / period
    return  seasonal_pattern(season_time) * amplitude

def noise(time, noise_level = 1, seed = None):
    rnd = np.random.RandomState(seed)
    return rnd.rand(len(time)) * noise_level

time = np.arange(4*365+1, dtype = 'float')
baseline = 10
amplitude = 40
slope = 0.2
noise_level = 5
series = baseline + trend(time,slope) + seasonality(time = time,period = 365, phase = 0,amplitude = amplitude) + noise(time, noise_level = noise_level, seed = 1)
test_test_split = 1000

time_train = time[:test_test_split]
series_train = series[:test_test_split]
time_test = time[test_test_split:]
series_test = series[test_test_split:]

window_size = 10
batch_size = 30
shuffle_batch_size = 1000
dataset2 = windowed_dataset(series_train,window_size,batch_size,shuffle_batch_size)

l0 = tf.keras.layers.Dense(1,input_shape = [window_size])
model = tf.keras.models.Sequential([l0])

lr_schedule = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-8 *10**(epoch/2000))
model.compile(loss = 'mse', optimizer= tf.keras.optimizers.SGD(lr=1e-6 ,momentum = 0.9))
history = model.fit(dataset2,epochs = 1001, steps_per_epoch=1, callbacks=[lr_schedule])

lrs = 1e-8 *10**(np.arange(1001)/2000)
plt.semilogx(lrs, history.history['loss'])

print(l0.get_weights())

forecast = []

for time in range(len(series) - window_size):
  forecast.append(model.predict(series[time:time + window_size][np.newaxis]))

forecast = forecast[test_test_split-window_size:]
results = np.array(forecast)[:, 0, 0]


plt.figure(figsize=(10, 6))

plot_series(time_test, series_test)
plot_series(time_test, results)

Week3: Lambda Layers

In [0]:
tf.keras.backend.clear_session()

model = tf.keras.models.Sequential([tf.keras.layers.Lambda(lambda x: tf.expand_dims(x,axis = -1), input_shape = [None]),
                                    tf.keras.layers.Bidirectional(tf.keras.layers.SimpleRNN(40, return_sequences=True)),
                                    tf.keras.layers.Bidirectional(tf.keras.layers.SimpleRNN(40)),
                                    tf.keras.layers.Dense(1),
                                    tf.keras.layers.Lambda(lambda x: x*10)])

Week4: dataset2 has got some issue, rest all is fine

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import sklearn
from matplotlib import style
style.use('seaborn')
tf.enable_eager_execution()

dataset = tf.data.Dataset.range(10)

for tensors in dataset:
    print(tensors)

dataset = dataset.window(5, shift = 1,stride = 1, drop_remainder = True)
for window_dataset in dataset:
    for val in window_dataset:
        print(val.numpy(), end = ',')
    print()

dataset = dataset.flat_map(lambda window: window.batch(5))
for window in dataset:
    print(window.numpy())

dataset = dataset.map(lambda window: (window[:-1],window[-1:]))
for x,y in dataset:
    print(x.numpy(),y.numpy())

dataset = dataset.shuffle(buffer_size = 10)
for x,y in dataset:
    print(x.numpy(),y.numpy())

dataset = dataset.batch(2).prefetch(1)
for x,y in dataset:
    print(x.numpy(),y.numpy())

def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    dataset = tf.data.Dataset.from_tensor_slices(series)
    dataset = dataset.window(window_size + 1, shift = 1,drop_remainder = True)
    dataset = dataset.flat_map(lambda window: window.batch(window_size+1))
    dataset = dataset.shuffle(shuffle_buffer)
    dataset = dataset.map(lambda window: (window[:-1],window[-1]))
    dataset = dataset.batch(batch_size).prefetch(1).repeat()
    return dataset

def model_forecast(model,series,window_size):
    dataset = tf.data.Dataset.from_tensor_slices(series)
    dataset = dataset.window(window_size, shift=1, drop_remainder=True)
    dataset = dataset.flat_map(lambda w: w.batch(window_size))
    dataset = dataset.batch(32).prefetch(1)
    return model.predict(dataset)

def plot_series(time, series, format = '-', start = 0 , end = None):
    plt.plot(time[start:end],series[start:end], format)
    plt.xlabel('time')
    plt.ylabel('values')
    plt.grid(True)

def trend(time, slope = 0):
    return slope*time

def seasonal_pattern(season_time):
    return np.where(season_time<0.4,
                    np.cos(season_time)*5,
                    np.exp(season_time))

def seasonality(time, period, phase = 0, amplitude = 1):
    season_time = ((time + phase) % period) / period
    return  seasonal_pattern(season_time) * amplitude

def noise(time, noise_level = 1, seed = None):
    rnd = np.random.RandomState(seed)
    return rnd.rand(len(time)) * noise_level

time = np.arange(4*365+1, dtype = 'float')
baseline = 10
amplitude = 40
slope = 0.2
noise_level = 5
series = baseline + trend(time,slope) + seasonality(time = time,period = 365, phase = 0,amplitude = amplitude) + noise(time, noise_level = noise_level, seed = 1)
test_test_split = 1000

time_train = time[:test_test_split]
series_train = series[:test_test_split]
time_test = time[test_test_split:]
series_test = series[test_test_split:]

window_size = 10
batch_size = 30
shuffle_batch_size = 1000
dataset2 = windowed_dataset(series_train,window_size,batch_size,shuffle_batch_size)

tf.keras.backend.clear_session()

model = tf.keras.models.Sequential([tf.keras.layers.Conv1D(filters=32, kernel_size=5,
                                                           strides=1, padding='causal',
                                                           activation='relu', input_shape=[None,1]),
                                    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),
                                    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),
                                    tf.keras.layers.Dense(1),
                                    tf.keras.layers.Lambda(lambda x: x*100)])

model.compile(loss = 'mse', optimizer= tf.keras.optimizers.SGD(lr=1e-5 ,momentum = 0.9))
model.fit(dataset2,epochs = 1001, steps_per_epoch=1)

rnn_forecast = model_forecast(model, series[..., np.newaxis], window_size)

In [0]:
#Sunspots model

model = tf.keras.models.Sequential([
  tf.keras.layers.Conv1D(filters=32, kernel_size=5,
                      strides=1, padding="causal",
                      activation="relu",
                      input_shape=[None, 1]),
  tf.keras.layers.LSTM(64, return_sequences=True),
  tf.keras.layers.LSTM(64, return_sequences=True),
  tf.keras.layers.Dense(30, activation="relu"),
  tf.keras.layers.Dense(10, activation="relu"),
  tf.keras.layers.Dense(1),
  tf.keras.layers.Lambda(lambda x: x * 400)
])